In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/final main.csv")
df.shape

(66397, 2)

In [4]:
df.shape

(66397, 2)

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66397 entries, 0 to 66396
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_info  66397 non-null  object
 1   desc          66397 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [7]:

import numpy as np
import re
import itertools
from collections import Counter
import pandas as pd
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


In [8]:
for i in range(0,df.shape[0]):
  df['desc'][i]= clean_str(df['desc'][i])

In [9]:
df.head()

,product_info,desc
0,SEER SECRETS ACTIVE SILVER ION DEODORANT CREAM,1mg body lotion with shea butter vitamin e p 1...
1,SEER SECRETS MILK BODY LOTION SPF 21,1mg body lotion with shea butter vitamin e p 1...
2,VITRO NATURALS ALOE SHEA BODY BUTTER,1mg body lotion with shea butter vitamin e p 1...
3,1MILE HEALTHCARE CLASSIC TONE STETHOSCOPE,1mile healthcare classic tone stethoscope orde...
4,1MILE HEALTHCARE DOCTOR PLUS STETHOSCOPE,1mile healthcare doctor plus stethoscope 606 o...


In [10]:
documents = df['desc'].values.astype("U")

In [11]:

vectorizer = TfidfVectorizer(stop_words='english')
features = vectorizer.fit_transform(documents)

In [ ]:
  # #ELBOW METHOD TO FIND K
# cost =[]
# for i in range(1000, 5000,500):
#     KM = KMeans(n_clusters = i, max_iter = 500)
#     KM.fit(features)
#     cost.append(KM.inertia_)     
  
# plt.plot(range(1000,5000), cost, color ='g', linewidth ='3')
# plt.xlabel("Value of K")
# plt.ylabel("Sqaured Error (Cost)")
# plt.show() 
  


In [ ]:
k = 1000
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(features)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=1000, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
import pickle
pickle.dump(model, open("/content/drive/My Drive/KMEANS CLUTERING with TF-IDF Vectorizer.sav", 'wb'))

In [ ]:
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)


In [12]:
import pickle
model= pickle.load(open("/content/drive/My Drive/KMEANS CLUTERING with TF-IDF Vectorizer.sav", 'rb'))

In [13]:
df['cluster'] = model.labels_

In [ ]:
df['cluster']

0         17
1         17
2         17
3        586
4        586
        ... 
66392    556
66393     83
66394    163
66395    460
66396     83
Name: cluster, Length: 66397, dtype: int32

In [14]:
df.head()

# output the result to a text file.

clusters = df.groupby('cluster')    

In [ ]:
k=1000
for cluster in clusters.groups:
    f = open('cluster'+str(cluster)+ '.csv', 'w') # create csv file
    data = clusters.get_group(cluster)[['product_info','desc']] # get title and overview columns
    f.write(data.to_csv(index_label='id')) # set index to id
    f.close()

print("Cluster centroids: \n")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(k):
    print("Cluster %d:" % i)
    for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
        print (' %s' % terms[j])
    print('------------')

In [60]:
def search(str):
  new_doc = [str]
  Y = vectorizer.transform(new_doc)
  prediction = model.predict(Y)
  print(prediction)
  result=pd.read_csv("cluster{}.csv".format(prediction[0]))
  indices = [i for i, s in enumerate(list(result["desc"])) if str in s]
  print(indices)
  if len(indices)==0:
    print(result["product_info"])
  else:
    for i in indices:
      print(result["product_info"][i])

In [61]:
search("sanitary pads")

[83]
[749, 1750, 2908, 2909, 3231, 4239, 6138, 6374, 6375, 6807, 6808, 7221, 7223, 7224, 7609, 7646, 7647, 7650, 7651, 7652, 7653]
SANNAP ADULT DIAPER DISPOSAL BAG
NECTAR ACNE SHINE MEDICATED SOAP
BELLA BABY HAPPY WET WIPES SENSITIVE ALOEVERA
BELLA COTTON PADS 80 30%
SANNAP PREMIUM SANITARY PADS ULTRA SILKY WINGS XL
CAREFREE PADS REGULAR
SANFE OXO BIODEGRADABLE DISPOSAL BAG
SOFY BODYFIT EXTRA LONG SANITARY PADS XL
SOFY BODYFIT OVERNIGHT SANITARY PADS XXL
WHISPER ULTRA CLEAN WINGS SANITARY PADS XL
WHISPER ULTRA CLEAN WINGS SANITARY PADS LARGE
SOFY ANTIBACTERIA SANITARY PADS XL
SOFY SOFT TAMPONS REGULAR
SOFY SOFT TAMPONS SUPER
VWASH WOW SANITARY NAPKIN ULTRA THIN XL
WHISPER CHOICE ULTRA SANITARY PADS XL
WHISPER CHOICE WINGS SANITARY PADS REGULAR
WHISPER ULTRA NIGHTS WINGS SANITARY PADS XL
WHISPER ULTRA NIGHTS WINGS SANITARY PADS XXXL
WHISPER ULTRA SOFT SANITARY PADS LARGE
WHISPER ULTRA SOFT SANITARY PADS XL
